In [ ]:
"""
Maxwell Zheng (1004907871)
Google Colab Link: 
"""

In [2]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

In [3]:
def get_relevant_indices(dataset, classes, target_classes):
    
    indices = []
    for i in range(len(dataset)):
        label_index = dataset[i][1]
        label_class = classes[label_index]
        if label_class in target_classes:
            indices.append(i)
            
    return indices

In [4]:
def get_data_loader(target_classes, batch_size):
    
    classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")
    
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    relevant_indices = get_relevant_indices(trainset, classes, target_classes)
    
    np.random.seed(1000)
    np.random.shuffle(relevant_indices)
    split = int(len(relevant_indices) * 0.8)
    
    relevant_train_indices, relevant_val_indices = relevant_indices[:split], relevant_indices[split:]
    train_sampler = SubsetRandomSampler(relevant_train_indices)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=1, sampler=train_sampler)
    val_sampler = SubsetRandomSampler(relevant_val_indices)
    val_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, num_workers=1, sampler=val_sampler)
    
    testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    relevant_test_indices = get_relevant_indices(testset, classes, target_classes)
    test_sampler = SubsetRandomSampler(relevant_test_indices)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, num_workers=1, sampler=test_sampler)
    
    return train_loader, val_loader, test_loader, classes

In [5]:
def get_model_name(name, batch_size, learning_rate, epoch):
    
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, learning_rate, epoch)
    
    return path

In [6]:
def normalize_label(labels):
    
    max_val = torch.max(labels)
    min_val = torch.min(labels)
    norm_labels = (labels - min_val)/(max_val - min_val)
    
    return norm_labels

In [7]:
def evaluate(net, loader, criterion):
    
    total_loss = 0.0
    total_err = 0.0
    total_epoch = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        labels = normalize_label(labels)
        outputs = net(inputs)
        loss = criterion(outputs, labels.float())
        corr = (outputs > 0.0).squeeze().long() != labels
        total_err += int(corr.sum())
        total_epoch += len(labels)
        total_loss += loss.item()
    
    err = float(total_err) / total_epoch
    loss = float(total_loss) / (i + 1)
    
    return err, loss

In [8]:
def plot_training_curve(path):
    
    import matplotlib.plyplot as plt
    
    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtext("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train as Validation Error")
    n = len(train_err)
    
    plt.plot(range(1,n+1), train_err, label="Train")
    plt.plot(range(1,n+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc="best")
    plt.show()
    
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc="best")
    plt.show()

In [9]:
train_loader, val_loader, test_loader, classes = get_data_loader(target_classes=["cat", "dog"], batch_size=1)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
# 1a

import matplotlib.pyplot as plt

k = 0
for images, labels in train_loader:
    image = images[0]
    img = np.transpose(image, [1,2,0])
    img = img / 2 + 0.5
    
    plt.subplot(3,5,k+1)
    plt.axis("off")
    plt.imshow(img)
    
    k += 1
    if k > 14:
        break

In [ ]:
#1b

"""
TODO
"""

In [ ]:
#1c

"""
TODO
"""

In [11]:
class LargeNet(nn.Module):
    
    def __init__(self):
        super(LargeNet, self).__init__()
        self.name = "large"
        self.conv1 = nn.Conv2d(3, 5, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 10, 5)
        self.fc1 = nn.Linear(10*5*5, 32)
        self.fc2 = nn.Linear(32, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 10*5*5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze(1)
        
        return x

In [12]:
class SmallNet(nn.Module):
    
    def __init__(self):
        super(SmallNet, self).__init__()
        self.name = "small"
        self.conv = nn.Conv2d(3, 5, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(5*7*7, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv(x)))
        x = self.pool(x)
        x = x.view(-1, 5*7*7)
        x = self.fc(x)
        x = x.squeeze(1)
        
        return x

In [13]:
small_net = SmallNet()
large_net = LargeNet()

In [19]:
def train_net(net, batch_size=64, learning_rate=0.01, num_epochs=30):
    
    target_classes = ["cat", "dog"]
    
    torch.manual_seed(1000)
    
    train_loader, val_loader, test_loader, classes = get_data_loader(target_classes, batch_size)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    
    train_err = np.zeros(num_epochs)
    train_loss = np.zeros(num_epochs)
    val_err = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    
    start_time = time.time()
    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_train_err = 0.0
        total_epoch = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            labels = normalize_label(labels)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            corr = (outputs > 0.0).squeeze().long() != labels
            total_train_err += int(corr.sum())
            total_train_loss += loss.item()
            total_epoch += len(labels)
        train_err[epoch] = float(total_train_err) / total_epoch
        train_loss[epoch] = float(total_train_loss) / total_epoch
        val_err[epoch], val_loss[epoch] = evaluate(net, val_loader, criterion)
        
        print(("Epoch {}: Train err: {}, Train loss: {} | Validation err: {}, Validation loss: {}").format(epoch+1, train_err[epoch], train_loss[epoch], val_err[epoch], val_loss[epoch]))
        
        model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)
    print("Finished Training")
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    
    epochs = np.arange(1, num_epochs+1)
    
    np.savetxt("{}_train_err.csv".format(model_path), train_err)
    np.savetxt("{}_train_loss.csv".format(model_path), train_loss)
    np.savetxt("{}_val_err.csv".format(model_path), val_err)
    np.savetxt("{}_val_loss.csv".format(model_path), val_loss)

In [15]:
print("Small")
for param in small_net.parameters():
    print(param.shape)
    
print("\nLarge")
for param in large_net.parameters():
    print(param.shape)

Small
torch.Size([5, 3, 3, 3])
torch.Size([5])
torch.Size([1, 245])
torch.Size([1])

Large
torch.Size([5, 3, 5, 5])
torch.Size([5])
torch.Size([10, 5, 5, 5])
torch.Size([10])
torch.Size([32, 250])
torch.Size([32])
torch.Size([1, 32])
torch.Size([1])


In [ ]:
# 2a

"""
TODO
"""

In [ ]:
# 2b

"""
Default values:
batch_size: 64
leaning_rate: 0.01
num_epochs: 30
"""

In [ ]:
# 2c
"""
Files written to disk:

1. model_small_bs64_lr0.01_epoch4_train_err.csv
    Info:
    4.359999999999999987e-01
    3.738750000000000129e-01
    3.578749999999999987e-01
    3.483749999999999902e-01
    3.385000000000000231e-01
2. model_small_bs64_lr0.01_epoch4_train_loss.csv
    Info:
    1.058855156600475335e-02
    1.014952868223190259e-02
    9.933517850935458596e-03
    9.757397778332232985e-03
    9.608423233032226812e-03
3. model_small_bs64_lr0.01_epoch4_val_err.csv
    Info
    3.805000000000000049e-01
    3.584999999999999853e-01
    3.524999999999999800e-01
    3.599999999999999867e-01
    3.370000000000000218e-01
4. model_small_bs64_lr0.01_epoch4_val_loss.csv
    Info:
    6.595180425792932510e-01
    6.528661046177148819e-01
    6.263249684125185013e-01
    6.251449063420295715e-01
    6.168369576334953308e-01
TODO
"""

In [ ]:
# 2d
"""
Small net: 347s
Large net: 394s
"""

In [18]:
train_net(small_net, num_epochs=5)

Files already downloaded and verified
Files already downloaded and verified
Epoch 1: Train err: 0.436, Train loss: 0.010588551566004753 | Validation err: 0.3805, Validation loss: 0.6595180425792933
Epoch 2: Train err: 0.373875, Train loss: 0.010149528682231903 | Validation err: 0.3585, Validation loss: 0.6528661046177149
Epoch 3: Train err: 0.357875, Train loss: 0.009933517850935459 | Validation err: 0.3525, Validation loss: 0.6263249684125185
Epoch 4: Train err: 0.348375, Train loss: 0.009757397778332233 | Validation err: 0.36, Validation loss: 0.6251449063420296
Epoch 5: Train err: 0.3385, Train loss: 0.009608423233032227 | Validation err: 0.337, Validation loss: 0.6168369576334953
Finished Training
Total time elapsed: 57.48 seconds
